In [1]:
!nvidia-smi

Fri Aug 21 14:32:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.92       Driver Version: 410.92       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:00:0D.0 Off |                  Off |
| N/A   67C    P0   184W / 250W |   7892MiB / 16130MiB |     65%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

online

if random.choice([0,1,1,1,]):
[0.859, 0.86, 0.861, 0.843, 0.855] 0.8556000000000001

[0.8789124971441612, 0.8831880936061866, 0.8818342151675468, 0.8683454177281323, 0.8759553203997628] 0.877647108809158

online 0.7916984126984127

In [2]:
seed = 0
import random
import numpy as np
import tensorflow as tf
import os
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
os.environ["PYTHONHASHSEED"] = str(seed)

os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [3]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=7000)])
    except RuntimeError as e:
        print(e)

In [4]:
import numpy as np
import pandas as pd
# 选择比较好的模型
import seaborn as sns

import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.signal import resample
from tensorflow.keras import layers
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import os
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from tensorflow.keras.losses import categorical_crossentropy

from sklearn.preprocessing import StandardScaler,MinMaxScaler
%load_ext autoreload
%autoreload 2

def acc_combo(y, y_pred):
    # 数值ID与行为编码的对应关系
    mapping = {
        '0': 'A_1', '1': 'B_2', '2': 'A_3', '3': 'A_4', '4': 'B_3', '5': 'C_5', '6': 'C_2', '7': 'A_5', '8': 'B_1', 
        '9': 'C_1', '10': 'A_2', '11': 'C_3', '12': 'B_5', '13': 'B_4', '14': 'C_4', 
        '15': 'D_6', '16': 'E_7', '17': 'F_8', '18': 'G_9', '19': 'H_0'
              }
    # 将行为ID转为编码
    code_y, code_y_pred = mapping[str(int(y))], mapping[str(int(y_pred))]
    if code_y == code_y_pred: #编码完全相同得分1.0
        return 1.0
    elif code_y.split("_")[0] == code_y_pred.split("_")[0]: #编码仅字母部分相同得分1.0/7
        return 1.0/7
    elif code_y.split("_")[1] == code_y_pred.split("_")[1]: #编码仅数字部分相同得分1.0/3
        return 1.0/3
    else:
        return 0.0



In [5]:
!ls ../data/final_data

sensor_test_final.csv  sensor_train_final.csv  submit_example.csv


In [6]:
root_path  = '../data/final_data/'
train = pd.read_csv(root_path+'sensor_train_final.csv')
test = pd.read_csv(root_path+'sensor_test_final.csv')
sub = pd.read_csv(root_path+'submit_example.csv')
y = train.groupby('fragment_id')['behavior_id'].min()

In [7]:
def add_features(df):
    print(df.columns)
    df['acc'] = (df.acc_x ** 2 + df.acc_y ** 2 + df.acc_z ** 2) ** .5
    df['accg'] = (df.acc_xg ** 2 + df.acc_yg ** 2 + df.acc_zg ** 2) ** .5
    df['thetax']=np.arctan(df.acc_xg/
                           np.sqrt(df.acc_yg*df.acc_yg+df.acc_zg*df.acc_zg))*180/np.pi
    df['thetay']=np.arctan(df.acc_yg/
                           np.sqrt(df.acc_xg*df.acc_xg+df.acc_zg*df.acc_zg))*180/np.pi
    df['thetaz']=np.arctan(df.acc_zg/
                           np.sqrt(df.acc_yg*df.acc_yg+df.acc_xg*df.acc_xg))*180/np.pi

    df['xy'] = (df['acc_x'] ** 2 + df['acc_y'] ** 2) ** 0.5
    df['xy_g'] = (df['acc_xg'] ** 2 + df['acc_yg'] ** 2) ** 0.5    
    
    df['g'] = ((df["acc_x"] - df["acc_xg"]) ** 2 + 
                 (df["acc_y"] - df["acc_yg"]) ** 2 + (df["acc_z"] - df["acc_zg"]) ** 2) ** 0.5

    print(df.columns)
    return df
train=add_features(train)
test=add_features(test)

Index(['fragment_id', 'time_point', 'acc_x', 'acc_y', 'acc_z', 'acc_xg',
       'acc_yg', 'acc_zg', 'behavior_id'],
      dtype='object')
Index(['fragment_id', 'time_point', 'acc_x', 'acc_y', 'acc_z', 'acc_xg',
       'acc_yg', 'acc_zg', 'behavior_id', 'acc', 'accg', 'thetax', 'thetay',
       'thetaz', 'xy', 'xy_g', 'g'],
      dtype='object')
Index(['fragment_id', 'time_point', 'acc_x', 'acc_y', 'acc_z', 'acc_xg',
       'acc_yg', 'acc_zg'],
      dtype='object')
Index(['fragment_id', 'time_point', 'acc_x', 'acc_y', 'acc_z', 'acc_xg',
       'acc_yg', 'acc_zg', 'acc', 'accg', 'thetax', 'thetay', 'thetaz', 'xy',
       'xy_g', 'g'],
      dtype='object')


In [8]:
group1 = [x for x in train.columns if x not in ['fragment_id', 'time_point','behavior_id']]
group1

['acc_x',
 'acc_y',
 'acc_z',
 'acc_xg',
 'acc_yg',
 'acc_zg',
 'acc',
 'accg',
 'thetax',
 'thetay',
 'thetaz',
 'xy',
 'xy_g',
 'g']

In [9]:
FEATURE_NUM=14

In [10]:
len(set(train['fragment_id'])),len(set(test['fragment_id']))

(15000, 16000)

In [11]:
sample_num = 30

In [12]:

x = np.zeros((15000, sample_num, FEATURE_NUM, 1))
t = np.zeros((16000, sample_num, FEATURE_NUM, 1))

In [13]:

train = train[['fragment_id', 'time_point', 'behavior_id']+group1]
test = test[['fragment_id', 'time_point']+group1]
print(train.columns)

for i in tqdm(range(7292)):
    tmp = train[train.fragment_id == i][:sample_num]
    x[i,:,:,0] = resample(tmp.drop(['fragment_id', 'time_point', 'behavior_id'],
                                    axis=1)[group1], sample_num, np.array(tmp.time_point))[0].reshape(sample_num,FEATURE_NUM)
for i in tqdm(range(7500)):
    tmp = test[test.fragment_id == i][:sample_num]
    
    
    
    t[i,:,:,0] = resample(tmp.drop(['fragment_id', 'time_point'],
                                    axis=1)[group1], sample_num, np.array(tmp.time_point))[0].reshape(sample_num,FEATURE_NUM)



  1%|          | 59/7292 [00:00<00:12, 588.53it/s]

Index(['fragment_id', 'time_point', 'behavior_id', 'acc_x', 'acc_y', 'acc_z',
       'acc_xg', 'acc_yg', 'acc_zg', 'acc', 'accg', 'thetax', 'thetay',
       'thetaz', 'xy', 'xy_g', 'g'],
      dtype='object')


100%|██████████| 7500/7500 [00:12<00:00, 593.02it/s]


In [14]:

train = train[['fragment_id', 'time_point', 'behavior_id']+group1]
test = test[['fragment_id', 'time_point']+group1]
print(train.columns)

for i in tqdm(range(7292)):
    tmp = train[train.fragment_id == i][:sample_num]
    x[i,:,:,0] = resample(tmp.drop(['fragment_id', 'time_point', 'behavior_id'],
                                    axis=1)[group1], sample_num, np.array(tmp.time_point))[0].reshape(sample_num,FEATURE_NUM)
for i in tqdm(range(7500)):
    tmp = test[test.fragment_id == i][:sample_num]
    t[i,:,:,0] = resample(tmp.drop(['fragment_id', 'time_point'],
                                    axis=1)[group1], sample_num, np.array(tmp.time_point))[0].reshape(sample_num,FEATURE_NUM)



  1%|          | 60/7292 [00:00<00:12, 599.11it/s]

Index(['fragment_id', 'time_point', 'behavior_id', 'acc_x', 'acc_y', 'acc_z',
       'acc_xg', 'acc_yg', 'acc_zg', 'acc', 'accg', 'thetax', 'thetay',
       'thetaz', 'xy', 'xy_g', 'g'],
      dtype='object')


100%|██████████| 7500/7500 [00:12<00:00, 589.48it/s]


In [15]:
# 一个完成了的generator
def data_generator(data,label,shuffle,batch_size):
    """
    data:array  (7292, 60, 14, 1)
    label:array (7292,)
    """
    
    count=0
    np.random.seed(seed)# 保证结果可重复
    length=len(data)
    while True:
        
        if count==0 or (count + 1) * batch_size > length:  # 如果是第一个或者最后一个batch
            count=0
            shuffle_index = list(range(length))
            np.random.shuffle(shuffle_index)   ## 对索引进行打乱
        
        start = count * batch_size  ## batch的起始点
        end = (count + 1) * batch_size ## batch的终点
        inds=shuffle_index[start:end]
        count+=1
#         print(count,inds)
        yield data[inds],label[inds]

count=0
for a,b in data_generator(x[:10],y[:10],True,3):
    print(a.shape,b.shape)
    count+=1
    if count==20:
        break

(3, 30, 14, 1) (3,)
(3, 30, 14, 1) (3,)
(3, 30, 14, 1) (3,)
(3, 30, 14, 1) (3,)
(3, 30, 14, 1) (3,)
(3, 30, 14, 1) (3,)
(3, 30, 14, 1) (3,)
(3, 30, 14, 1) (3,)
(3, 30, 14, 1) (3,)
(3, 30, 14, 1) (3,)
(3, 30, 14, 1) (3,)
(3, 30, 14, 1) (3,)
(3, 30, 14, 1) (3,)
(3, 30, 14, 1) (3,)
(3, 30, 14, 1) (3,)
(3, 30, 14, 1) (3,)
(3, 30, 14, 1) (3,)
(3, 30, 14, 1) (3,)
(3, 30, 14, 1) (3,)
(3, 30, 14, 1) (3,)


In [16]:
def ConvBNRelu(X,filters,kernal_size=(3,3)):
    X = Conv2D(filters=filters,
               kernel_size=kernal_size,
#                activation='relu',
               use_bias=False,
               padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    return X


def ConvRelu(X,filters,kernal_size=(3,3)):
    X = Conv2D(filters=filters,
               kernel_size=kernal_size,
               activation='relu',
               use_bias=False,
               padding='same')(X)
    return X


def squeeze_excitation_layer(x, out_dim,ratio=8):
    '''
    SE module performs inter-channel weighting.
    '''
    squeeze = GlobalAveragePooling2D()(x)

    excitation = Dense(units=out_dim // ratio)(squeeze)
    excitation = Activation('relu')(excitation)
    excitation = Dense(units=out_dim)(excitation)
    excitation = Activation('sigmoid')(excitation)
    excitation = Reshape((1,1,out_dim))(excitation)
    scale = multiply([x,excitation])
    return scale

# def SE_Residual(X):
#     A = 
#     X = squeeze_excitation_layer(X,128)
#     X =  Add()([X,A])
    

def lenet5(input):
    A = ConvBNRelu(input,64,kernal_size=(3,3))
#     B = ConvBNRelu(input,16,kernal_size=(5,1))
#     C = ConvBNRelu(input,16,kernal_size=(7,1))
#     ABC = layers.Concatenate()([A,B,C])
    X = ConvBNRelu(A,128)
#     X = squeeze_excitation_layer(X,128)
    X = Dropout(0.2)(X)

    X = AveragePooling2D()(X)
    
    X = ConvBNRelu(X,256)
    X = Dropout(0.3)(X)
#     X = squeeze_excitation_layer(X,256)
    X = ConvBNRelu(X,512)   
    X = Dropout(0.5)(X)
#     X = squeeze_excitation_layer(X,512)
#     X = GlobalMaxPooling2D()(X)
    X = GlobalAveragePooling2D()(X)
    
#     X = BatchNormalization()(X)
    return X
import tensorflow as tf
def Net(sample_num):
    input1 = Input(shape=(sample_num, FEATURE_NUM, 1))
    part = tf.split(input1,axis=2, num_or_size_splits = [6, 2, 6])
#     res = tf.split(c, axis = 3, num_or_size_splits = [2, 2, 4])
    
    
    X1 = Concatenate(axis=-2)([part[0],part[1]])
    X1 = lenet5(X1)
    X1 = BatchNormalization()(X1)
    X1 = Dense(128, activation='relu')(X1)
    X1 = BatchNormalization()(X1)
    X1 = Dropout(0.2)(X1)

    X2 = Concatenate(axis=-2)([part[0],part[2]])
    X2 = lenet5(X2)    
    X2 = BatchNormalization()(X2)
#     X = Dense(512, activation='relu')(X)
#     X = BatchNormalization()(X)
    X2 = Dense(128, activation='relu')(X2)
    X2 = BatchNormalization()(X2)
    X2 = Dropout(0.2)(X2)
    
    X = Concatenate(axis=-1)([X1,X2])
    
#     X = Dense(256)(X)    
    
#     output1 = Dense(4, activation='softmax', name='4class')(X)   # 大类-字母
#     output2 = Dense(128)(X)
#     output2 = Dense(64)(X)
#     X = Dense(64)(X)
#     output2 = Dense(7, activation='softmax', name='7class')(X)   # 大类-数字
#     X = Dense(32)(X)
#     X = Concatenate(axis=-1)([X,output1,output2])
    X = Dense(64)(X)
    output3 = Dense(20, activation='softmax',name='20class')(X) #小类
    
    
    return Model([input1], [output3])

# model = Net(60)
# model.summary()

In [17]:


# [:,:,:,[1]]
train = x
test = t

y_binary = to_categorical(y)

fold_num=3

proba_t = np.zeros((15000, 20))
proba_oof = np.zeros((16000,20))

oof_score = []
oof_comm = []
history = []

seeds=[42]
for seed in seeds:
    kfold = StratifiedKFold(fold_num,random_state=seed,shuffle=True)
    
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    
    for fold, (xx, yy) in enumerate(kfold.split(train, y)):


        model = Net(30)
        model.summary()
        model.compile(loss=categorical_crossentropy,
                      optimizer=Adam(),
                      metrics=["acc"])#'',localscore

        plateau3 = ReduceLROnPlateau(monitor="acc",
                                    verbose=1,
                                    mode='max',
                                    factor=0.1,
                                    patience=18)
        early_stopping = EarlyStopping(monitor="val_acc",
                                       verbose=1,
                                       mode='max',
                                       patience=60)

        checkpoint = ModelCheckpoint(f'bl{fold}.h5',
                                     monitor="val_acc",
                                     verbose=0,
                                     mode='max',
                                     save_best_only=True)

        train_res = model.fit(train[xx],y_binary[xx],batch_size=32,
                  epochs=1000,shuffle=True,
                  validation_data=(train[yy],y_binary[yy]),
                    callbacks=[plateau3, early_stopping, checkpoint])
        history.append(train_res)

        model.load_weights(f'bl{fold}.h5')
        proba_t += model.predict(test, verbose=0, batch_size=1024)[2] / (fold_num*len(seeds))
        proba_oof[yy] += model.predict(train[yy],verbose=0,batch_size=1024) [2]

        oof_y = np.argmax(proba_oof[yy], axis=1)
        acc = round(accuracy_score(y[yy], oof_y),3)
        print(acc)
        oof_score.append(acc)
        scores = sum(acc_combo(y_true, y_pred) for y_true, y_pred in zip(y[yy], oof_y)) / oof_y.shape[0]
        oof_comm.append(scores)   
        print(round(scores, 5))


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30, 14, 1)]  0                                            
__________________________________________________________________________________________________
tf_op_layer_split (TensorFlowOp [(None, 30, 6, 1), ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 30, 8, 1)     0           tf_op_layer_split[0][0]          
                                                                 tf_op_layer_split[0][1]          
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 30, 12, 1)    0           tf_op_layer_split[0][0]      

10000/10000 [==============================] - 16s 2ms/sample - loss: 2.7835 - acc: 0.1692 - val_loss: 2.7093 - val_acc: 0.1712
Epoch 2/1000
10000/10000 [==============================] - 7s 711us/sample - loss: 2.5426 - acc: 0.2095 - val_loss: 2.5469 - val_acc: 0.2158
Epoch 3/1000
10000/10000 [==============================] - 7s 698us/sample - loss: 2.4547 - acc: 0.2285 - val_loss: 2.3030 - val_acc: 0.2660
Epoch 4/1000
10000/10000 [==============================] - 7s 682us/sample - loss: 2.4030 - acc: 0.2433 - val_loss: 2.4005 - val_acc: 0.2426
Epoch 5/1000
10000/10000 [==============================] - 7s 695us/sample - loss: 2.3584 - acc: 0.2517 - val_loss: 2.2518 - val_acc: 0.2884
Epoch 6/1000
10000/10000 [==============================] - 4s 427us/sample - loss: 2.3201 - acc: 0.2694 - val_loss: 2.2515 - val_acc: 0.2744
Epoch 7/1000
10000/10000 [==============================] - 3s 345us/sample - loss: 2.2899 - acc: 0.2697 - val_loss: 2.2347 - val_acc: 0.2822
Epoch 8/1000
10000/1

10000/10000 [==============================] - 7s 739us/sample - loss: 1.9131 - acc: 0.3884 - val_loss: 1.9946 - val_acc: 0.3784
Epoch 59/1000
10000/10000 [==============================] - 7s 738us/sample - loss: 1.9154 - acc: 0.3984 - val_loss: 1.9953 - val_acc: 0.3758
Epoch 60/1000
10000/10000 [==============================] - 7s 731us/sample - loss: 1.9050 - acc: 0.3950 - val_loss: 2.0251 - val_acc: 0.3712
Epoch 61/1000
10000/10000 [==============================] - 7s 730us/sample - loss: 1.9018 - acc: 0.3957 - val_loss: 1.9881 - val_acc: 0.3792
Epoch 62/1000
10000/10000 [==============================] - 7s 732us/sample - loss: 1.8905 - acc: 0.3980 - val_loss: 1.9957 - val_acc: 0.3794
Epoch 63/1000
10000/10000 [==============================] - 7s 695us/sample - loss: 1.8836 - acc: 0.4015 - val_loss: 2.0302 - val_acc: 0.3740
Epoch 64/1000
10000/10000 [==============================] - 7s 729us/sample - loss: 1.8946 - acc: 0.3957 - val_loss: 2.0072 - val_acc: 0.3788
Epoch 65/1000

KeyboardInterrupt: 